# Proof of concept of AutoCode: Automatic code selection based on metric(s) optimization.

Example: Computational web app to be optimized by sum of variable values and latency.


In [3]:
from __future__ import annotations

import math

import nest_asyncio

nest_asyncio.apply()


def a_0(data):
    count = 0
    for i in range(1, len(data)):
        count += 1

    return count


def a_1(data):
    count = 0
    for i in range(1, len(data)):
        for j in range(1, len(data)):
            count += 1

    return count


def b_0(data):
    count = 0
    for i in range(1, int(len(data) / 2)):
        count += 1

    return count


def b_1(data):
    count = 0
    for i in range(1, int(len(data) / 2)):
        for j in range(1, int(len(data) / i)):
            count += 1

    return count


def b_2(data):
    count = 0
    for i in range(1, len(data)):
        for j in range(1, len(data)):
            for k in range(1, len(data)):
                count += 1

    return count


def c_0(data):
    count = 0
    for i in range(1, len(data)):
        for j in range(1, i ** 2):
            count += 1

    return count


def c_1(data):
    count = 0
    for i in range(1, int(math.log(len(data), 2))):
        for j in range(1, len(data) - i):
            count += 1

    return count

In [1]:
import tensorflow
from tensorflow.compiler.tf2tensorrt._pywrap_py_utils import get_linked_tensorrt_version, get_loaded_tensorrt_version

print(f"Linked TensorRT version {get_linked_tensorrt_version()}")
print(f"Loaded TensorRT version {get_loaded_tensorrt_version()}")

print(tensorflow.config.list_physical_devices('GPU'))

import torch

print("Is CUDA available: ", torch.cuda.is_available())
print(f"Available CUDA devices: {torch.cuda.device_count()}")

2024-07-02 19:26:50.649672: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 19:26:50.674552: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 19:26:51.207907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-02 19:26:51.873294: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 19:26:51.896287: 

[]
Is CUDA available:  True
Available CUDA devices: 1


In [10]:
from distributed import LocalCluster
from dask.distributed import Client

# cluster = LocalCUDACluster()
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39757,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:41215,Total threads: 1
Dashboard: http://127.0.0.1:42501/status,Memory: 495.74 MiB
Nanny: tcp://127.0.0.1:44453,


In [9]:
client.shutdown()

2024-07-04 21:57:02,021 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/usr/local/lib/python3.10/dist-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/usr/local/lib/python3.10/dist-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/usr/local/lib/python3.10/dist-packages/distributed/core.py", line 1395, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwarg

In [2]:
import ray

ray.shutdown()
ray.init(dashboard_host="0.0.0.0")
ray.available_resources()

2024-07-04 21:22:58,763	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66322432 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.33gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-07-04 21:22:58,955	INFO worker.py:1762 -- Started a local Ray instance. View the dashboard at 172.23.0.2:8265 


{'accelerator_type:G': 1.0,
 'node:__internal_head__': 1.0,
 'node:172.23.0.2': 1.0,
 'CPU': 32.0,
 'memory': 6508545639.0,
 'object_store_memory': 3254272819.0,
 'GPU': 1.0}

In [10]:
import httpx
import os
import dask
from fastapi import FastAPI


async def root():
    return {"message": "Hello World"}


queue = dask.distributed.Queue()
for i in range(os.cpu_count()):
    # app = i
    app = FastAPI()
    app.i = i
    app.add_api_route("/", root, methods=["GET"])
    queue.put(app)


def task():
    v = queue.get()
    client = httpx.AsyncClient(app=v, base_url="http://run")
    response_future = client.get("/")
    response = asyncio.run(response_future)
    print(response.json())
    queue.put(v)
    return v


futures = [client.submit(task) for i in range(5000)]
for future in dask.distributed.as_completed(futures):
    val = future.result()
    print(val.i)
    # print(val)    

2024-07-04 03:51:09,817 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/distributed/protocol/core.py", line 175, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/usr/local/lib/python3.10/dist-packages/distributed/protocol/core.py", line 159, in _decode_default
    return merge_and_deserialize(
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/distributed/protocol/serialize.py", line 522, in merge_and_deserialize
    return deserialize(header, merged_frames, deserializers=deserializers)
  File "/usr/local/lib/python3.10/dist-packages/distributed/protocol/serialize.py", line 444, in deserialize
    raise TypeError(
TypeError: Data serialized with dask but only able to deserialize data with ['pickle']


TypeError: Data serialized with dask but only able to deserialize data with ['pickle']

In [9]:
from dask.distributed import get_client, as_completed

d1 = [get_client().submit(c_0, [i] * i) for i in range(500)]
count = 0
for d in as_completed(d1):
    count += 1
    print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
from concurrent import futures
import ray

r = ray.remote(c_0)
# r = r.options(num_gpus=1, num_cpus=24)
r_futures = [r.remote([i] * i).future() for i in range(500)]
count = 0
for rs_future in futures.as_completed(r_futures):
    count += 1
    print(count)



2024-07-12 15:32:05,410	INFO worker.py:1762 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
from concurrent import futures

with futures.ProcessPoolExecutor(max_workers=24) as executor:
    future_list = [executor.submit(c_0, [i] * i) for i in range(500)]
    count = 0
    for future in futures.as_completed(future_list):
        count += 1
        print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
# from ray.util.queue import Queue
from dask.distributed import Queue, get_client
import psutil
from abc import ABC, abstractmethod
from pymoo.core.result import Result
import asyncio
from datetime import datetime
from httpx import AsyncClient
from typing import Any, Dict, Optional, Callable, Union, Tuple
from pymoo.core.variable import Real, Integer, Choice, Binary
from fastapi import FastAPI
from pymoo.algorithms.moo.nsga2 import RankAndCrowding
from pymoo.core.mixed import MixedVariableGA
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
import pandas as pd
from pymoo.visualization.pcp import PCP
from pymoo.visualization.scatter import Scatter
from pymoo.mcdm.pseudo_weights import PseudoWeights


class AutoCodeObjective:

    def __init__(
            self,
            type: str,
            weight: float
    ):
        self.type: str = type
        self.weight: float = weight


class AutoCodeVariable:

    def __init__(
            self,
            id: str = None
    ):
        self.id: Optional[str] = id


class AutoCodeServerFunction:
    def __init__(
            self,
            id: str,
            name: str,
    ):
        self.id: str = id
        self.name: str = name


class AutoCodeClientFunction:
    def __init__(
            self,
            function: Callable,
            function_args: Callable[[Optional[AutoCodeClientApplication]], List[Any]] = None,
            function_kwargs: Callable[[Optional[AutoCodeClientApplication]], Dict[str, Any]] = None
    ):
        self.id: str = str(uuid.uuid4())
        self.function: Callable = function
        self.function_args: Callable[[Optional[AutoCodeClientApplication]], List[Any]] = function_args or {}
        self.function_kwargs: Callable[[Optional[AutoCodeClientApplication]], Dict[str, Any]] = function_kwargs or {}

    def execute(self, application: AutoCodeClientApplication) -> Any:
        value: Any = self.function(
            *self.function_args(application),
            **self.function_kwargs(application),
        )

        return value


class AutoCodeServerChoice(AutoCodeVariable):

    def __init__(
            self,
            options: List[Any | AutoCodeServerFunction],
            *args,
            **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.options: List[Any | AutoCodeServerFunction] = options


class AutoCodeClientChoice(AutoCodeVariable):

    def __init__(
            self,
            options: List[Any | AutoCodeClientFunction],
            *args,
            **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.options: List[Any | AutoCodeClientFunction] = options


class AutoCodeBinary(AutoCodeVariable):
    def __init__(
            self,
            *args,
            **kwargs
    ):
        super().__init__(*args, **kwargs)


class AutoCodeInteger(AutoCodeVariable):
    def __init__(
            self,
            bounds: Tuple[int, int],
            *args,
            **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.bounds: Tuple[int, int] = bounds


class AutoCodeReal(AutoCodeVariable):
    def __init__(
            self,
            bounds: Tuple[float, float],
            *args,
            **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.bounds: Tuple[float, float] = bounds


class AutoCodeResult:
    def __init__(
            self,
            optimization: Result,
            decision_index: Integer,
            decision_solution: Any,
            decision_objective: Any
    ):
        self.optimization: Result = optimization
        self.decision_index: Integer = decision_index
        self.decision_solution: Any = decision_solution
        self.decision_objective: Any = decision_objective


class AutoCodeEvaluationResult:
    def __init__(
            self,
            objectives: List[Any],
            inequality_constraints: Optional[List[Any]] = None,
            equality_constraints: Optional[List[Any]] = None,
    ):
        self.objectives: List[Any] = objectives
        self.inequality_constraints: List[Any] = inequality_constraints
        self.equality_constraints: List[Any] = equality_constraints


class AutoCodeServer:
    def __init__(
            self,
            objectives: List[AutoCodeObjective],
            variables: Dict[str, Union[AutoCodeBinary, AutoCodeServerChoice, AutoCodeInteger, AutoCodeReal]],
            client: AutoCodeClient
    ):
        self.num_workers: int = psutil.cpu_count(logical=True)
        self.application_ids: List[str] = [str(uuid.uuid4()) for _ in range(self.num_workers)]
        self.objectives: List[AutoCodeObjective] = objectives
        self.variables: Dict[str, AutoCodeBinary | AutoCodeServerChoice | AutoCodeInteger | AutoCodeReal] = variables
        self.client: AutoCodeClient = client

    def run(self):
        optimization: Result = self.minimize()
        decision_index: Integer = self.get_decision_index(optimization)

        for index, objective in enumerate(self.objectives):
            if objective.type == "maximize":
                optimization.F[:, index] = -optimization.F[:, index]

        if optimization.problem.n_obj == 1:
            optimization.X = np.array([optimization.X])

        result: AutoCodeResult = AutoCodeResult(
            optimization=optimization,
            decision_index=decision_index,
            decision_solution=optimization.X[decision_index],
            decision_objective=optimization.F[decision_index]
        )

        return result

    def minimize(self) -> Result:
        runner: AutoCodeProblemRunner = AutoCodeProblemRunner()

        problem: AutoCodeProblem = AutoCodeProblem(
            auto_code=self,
            elementwise_runner=runner
        )

        algorithm: MixedVariableGA = MixedVariableGA(
            survival=RankAndCrowding()
        )

        result: Result = minimize(
            problem,
            algorithm,
            seed=1,
            verbose=True
        )

        return result

    def get_decision_index(self, result: Result) -> Integer:
        un_normalized_weights = np.array([objective.weight for objective in self.objectives])
        normalized_weights = un_normalized_weights / np.sum(un_normalized_weights)

        mcdm = PseudoWeights(normalized_weights)
        decision_index = mcdm.do(result.F)

        return decision_index

    def plot(self, result: AutoCodeResult):
        pass


class AutoCodeProblemRunner:
    def __init__(self):
        pass

    # def __call__(self, f, X):
    #     runnable = ray.remote(f.__call__.__func__)
    #     futures = [runnable.remote(f, x) for x in X]
    #     return ray.get(futures)

    def __call__(self, f, X):
        task_futures = [get_client().submit(f, x) for x in X]
        results = get_client().gather(task_futures)
        return results

    # def __call__(self, f, X):
    #     futures = [f(x) for x in X]
    #     return futures

    def __getstate__(self):
        state = self.__dict__.copy()
        return state


class AutoCodeProblem(ElementwiseProblem):

    def __init__(
            self,
            auto_code: AutoCodeServer,
            *args,
            **kwargs
    ):
        self.auto_code = auto_code
        self.queue: Queue = Queue()
        for application_id in self.auto_code.application_ids:
            self.queue.put(application_id)

        self.vars = {}
        for variable in self.auto_code.variables.values():
            if variable.__class__.__name__ == AutoCodeBinary.__name__:
                self.vars[variable.id] = Binary()
            elif variable.__class__.__name__ == AutoCodeServerChoice.__name__:
                self.vars[variable.id] = Choice(options=variable.options)
            elif variable.__class__.__name__ == AutoCodeInteger.__name__:
                self.vars[variable.id] = Integer(bounds=variable.bounds)
            elif variable.__class__.__name__ == AutoCodeReal.__name__:
                self.vars[variable.id] = Real(bounds=variable.bounds)
            else:
                raise ValueError(f"Type of variable with id {variable.id} is not supported.")

        super().__init__(
            vars=self.vars,
            n_obj=len(self.auto_code.objectives),
            *args,
            **kwargs
        )

    def _evaluate(self, X, out, *args, **kwargs):
        print(X)
        # application_id: str = self.queue.get()
        # evaluation: AutoCodeServerEvaluation = AutoCodeServerEvaluation(
        #     application_id=application_id,
        #     variables=X
        # )
        # print(evaluation.__dict__)
        # # result: AutoCodeEvaluationResult = self.auto_code.client.evaluate(evaluation)
        # self.queue.put(application_id)
        # 
        # if len(result.objectives) != self.n_obj:
        #     raise ValueError(
        #         "The number of evaluated objectives does not match to predefined objectives."
        #     )
        # 
        # for index, objective in enumerate(self.auto_code.objectives):
        #     if objective.type == "maximize":
        #         result.objectives[index] = -result.objectives[index]
        # 
        # out["F"] = result.objectives
        # 
        # if result.inequality_constraints is not None:
        #     if len(result.inequality_constraints) != self.n_ieq_constr:
        #         raise ValueError(
        #             "The number of evaluated inequality constraints does not match to predefined inequality constraints."
        #         )
        # 
        #     out["G"] = result.inequality_constraints
        # 
        # if result.equality_constraints is not None:
        #     if len(result.equality_constraints) != self.n_eq_constr:
        #         raise ValueError(
        #             "The number of evaluated equality constraints does not match to predefined equality constraints."
        #         )
        # 
        #     out["H"] = result.equality_constraints


class AutoCodeServerEvaluation:

    def __init__(
            self,
            application_id: str,
            variables: Dict[str, Union[Any | AutoCodeServerFunction]]
    ):
        self.application_id: str = application_id
        self.variables: Dict[str, Union[Any | AutoCodeServerFunction]] = variables


class AutoCodeClientEvaluation:

    def __init__(
            self,
            variables: Dict[str, Union[Any | AutoCodeClientFunction]]
    ):
        self.variables: Dict[str, Union[Any | AutoCodeClientFunction]] = variables
        self.executed_variables: Dict[str, Any] = {}


class AutoCodeClient:
    def __init__(
            self,
            objectives: List[AutoCodeObjective],
            variables: List[AutoCodeBinary | AutoCodeClientChoice | AutoCodeInteger | AutoCodeReal],
            evaluator: Callable[[AutoCodeClientApplication], AutoCodeEvaluationResult],
            application_class: Callable[..., AutoCodeClientApplication],
    ):
        self.objectives: List[AutoCodeObjective] = objectives
        self.evaluator: Callable[[AutoCodeClientApplication], AutoCodeEvaluationResult] = evaluator
        self.variables: Dict[str, Union[AutoCodeBinary, AutoCodeClientChoice, AutoCodeInteger, AutoCodeReal]] = {}
        server_variables: Dict[str, AutoCodeBinary | AutoCodeServerChoice | AutoCodeInteger | AutoCodeReal] = {}
        for variable in variables:
            if variable.id in self.variables:
                raise ValueError(f"Variable with id {variable.id} is already defined.")

            self.variables[variable.id] = variable

            if variable.__class__.__name__ == AutoCodeClientChoice.__name__:
                server_options: List[AutoCodeServerFunction] = []
                for option in variable.options:
                    if option.__class__.__name__ == AutoCodeClientFunction.__name__:
                        server_option = AutoCodeServerFunction(
                            id=option.id,
                            name=f"{option.function.__name__}"
                        )
                        server_options.append(server_option)
                    else:
                        server_option = option
                        server_options.append(server_option)
                server_variable = AutoCodeServerChoice(
                    id=variable.id,
                    options=server_options
                )
                server_variables[variable.id] = server_variable
            else:
                server_variables[variable.id] = variable

        self.server: AutoCodeServer = AutoCodeServer(
            objectives=objectives,
            variables=server_variables,
            client=self
        )

        self.applications: Dict[str, AutoCodeClientApplication] = {}
        for application_id in self.server.application_ids:
            application = application_class()
            application.auto_code = self
            application.id = application_id
            self.applications[application_id] = application

    def evaluate(self, evaluation: AutoCodeServerEvaluation) -> AutoCodeEvaluationResult:
        client_variables = {}
        for variable_id, value in evaluation.variables.items():
            if value.__class__.__name__ == AutoCodeServerFunction.__name__:
                for client_option in self.variables[variable_id].options:
                    if client_option.id == value.id:
                        client_variables[variable_id] = client_option
            else:
                client_variables[variable_id] = value
        client_evaluation: AutoCodeClientEvaluation = AutoCodeClientEvaluation(
            variables=client_variables
        )
        application: AutoCodeClientApplication = self.applications[evaluation.application_id]
        application.evaluation = client_evaluation
        result: AutoCodeEvaluationResult = self.evaluator(application)
        return result

    def run(self) -> AutoCodeResult:
        result: AutoCodeResult = self.server.run()

        return result


class AutoCodeClientApplication(ABC):

    def __init__(self):
        self.id: Optional[str] = None
        self.auto_code: Optional[AutoCodeClient] = None
        self.evaluation: Optional[AutoCodeClientEvaluation] = None

    def get_value(self, variable_id: str) -> Any:
        if variable_id not in self.evaluation.executed_variables:
            value: Union[Any | AutoCodeClientFunction] = self.evaluation.variables[variable_id]

            if value.__class__.__name__ == AutoCodeClientFunction.__name__:
                value = value.execute(self)

            self.evaluation.executed_variables[variable_id] = value

        return self.evaluation.executed_variables[variable_id]

    @abstractmethod
    def run(self) -> None:
        pass


In [12]:
import uuid


class Application(AutoCodeClientApplication):
    def __init__(self):
        super().__init__()
        self.app: FastAPI = FastAPI()
        self.app.add_api_route("/", self.root, methods=["GET"])

    def run(self):
        client: AsyncClient = AsyncClient(
            app=self.app,
            base_url=f"http://{uuid.uuid4()}",
        )
        # response_future = client.get("/")
        # asyncio.run(response_future)
        for route in self.app.routes:
            for method in route.methods:
                response_future = client.request(
                    method=method,
                    url=route.path
                )
                asyncio.run(response_future)

    async def root(self):
        a = self.get_value("a")

        b = self.get_value("b")

        c = self.get_value("c")

        d = int(self.get_value("d"))

        e = float(self.get_value("e"))

        f = bool(self.get_value("f"))

        g = ((d * e) % 2) == 0

        h = g and f

        result = {
            "a": a,
            "b": b,
            "c": c,
            "d": d,
            "e": e,
            "f": f,
            "g": g,
            "h": h
        }

        return result


def evaluator(application: AutoCodeClientApplication) -> AutoCodeEvaluationResult:
    t0 = datetime.now()
    application.run()
    t1 = datetime.now()

    f_latency = t1 - t0
    f_latency = f_latency.microseconds

    f_executed_sum = sum(application.evaluation.executed_variables.values())

    result = AutoCodeEvaluationResult(
        objectives=[f_latency, f_executed_sum]
    )

    return result


auto_code_client: AutoCodeClient = AutoCodeClient(
    objectives=[
        AutoCodeObjective(
            type="minimize",
            weight=1
        ),
        AutoCodeObjective(
            type="maximize",
            weight=1
        ),
    ],
    variables=[
        AutoCodeClientChoice(
            id="a",
            options=[
                AutoCodeClientFunction(
                    function=a_0,
                    function_args=lambda application: [[0] * 2500],
                    function_kwargs=lambda application: {}
                ),
                AutoCodeClientFunction(
                    function=a_1,
                    function_args=lambda application: [[0] * 2500],
                    function_kwargs=lambda application: {}
                ),
            ]
        ),
        AutoCodeClientChoice(
            id="b",
            options=[
                AutoCodeClientFunction(
                    function=b_0,
                    function_args=lambda application: [[0] * application.get_value("a")],
                    function_kwargs=lambda application: {}
                ),
                AutoCodeClientFunction(
                    function=b_1,
                    function_args=lambda application: [[0] * application.get_value("a")],
                    function_kwargs=lambda application: {}
                ),
                AutoCodeClientFunction(
                    function=b_2,
                    function_args=lambda application: [[0] * 1],
                    function_kwargs=lambda application: {}
                ),
            ]
        ),
        AutoCodeClientChoice(
            id="c",
            options=[
                AutoCodeClientFunction(
                    function=c_0,
                    function_args=lambda application: [[0] * 1],
                    function_kwargs=lambda application: {}
                ),
                AutoCodeClientFunction(
                    function=c_1,
                    function_args=lambda application: [[0] * 1],
                    function_kwargs=lambda application: {}
                ),
            ]
        ),
        AutoCodeInteger(
            id="d",
            bounds=(-10, 10)
        ),
        AutoCodeReal(
            id="e",
            bounds=(-3.14, 3.14)
        ),
        AutoCodeBinary(
            id="f"
        )
    ],
    evaluator=evaluator,
    application_class=Application
)


In [13]:
result = auto_code_client.run()

Exception: RuntimeError('Error during deserialization of the task graph. This frequently\noccurs if the Scheduler and Client have different environments.\nFor more information, see\nhttps://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments\n')

In [ ]:
pd.set_option('display.max_colwidth', None)

plot_0 = Scatter()
plot_0.add(result.optimization.F, color="blue")
plot_0.add(result.decision_objective, color="green")
plot_0.title = "Objective Space"
plot_0.show()

plot_1 = PCP()
plot_1.add(result.optimization.F, color="blue")
plot_1.add(result.decision_objective, color="green")
plot_1.title = "Objective Space"
plot_1.show()

plot_2_maps = []
for index, (x, f) in enumerate(zip(result.optimization.X, result.optimization.F)):
    x_values = {}
    for variable_id, value in x.items():
        if value.__class__.__name__ == AutoCodeServerFunction.__name__:
            x_values[variable_id] = value.name
        else:
            x_values[variable_id] = value

    f_values = {}
    for objective_index, value in enumerate(f):
        f_values[objective_index + 1] = value

    plot_2_map = {
        "x": x_values,
        "f": f_values,
        "decision": index == result.decision_index,
    }
    plot_2_maps.append(plot_2_map)

plot_2_df = pd.DataFrame(plot_2_maps)
plot_2_df


In [10]:

from playground.inter_languages.server.model import BaseModelV2
from typing import List


class A(BaseModelV2):
    a: str
    b: str
    
    def __init__(self, **kwargs):
        print(type(self))
        super().__init__(**kwargs)

    def x(self):
        return type(self).__name__
    
    def y(self):
        return self.c


class B(A):
    c: str
    
class C(B):
    d: str


a1 = A(a="a", b="b")
a2 = A(a="a", b="b")

b1 = B(a="a", b="b", c="c")
c1 = C(a="a", b="b", c="c", d="d")

c1.json()  == str(c1.model_dump_json())

# print(b.model_dump_json())

<class '__main__.A'>
<class '__main__.A'>
<class '__main__.B'>
<class '__main__.C'>


True

In [20]:
d = {A: a1}
d[type(a1)].__class__.__name__

'A'

In [1]:
print(type('a'))

<class 'str'>


In [29]:
x[0]

array([1, 2, 3])

In [56]:
b1.y()

'c'